In [29]:
import glob
import os
import re
from functools import cached_property

import pandas as pd

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 1000)

The data is separated into multiple files - one per each month.

First let's learn how to parse a single file:

In [27]:
class BankStatement:
    
    def __init__(self, filepath: str):
        self.df = pd.read_excel(filepath, header=None)
        self.s = self.df[0]
        self.s.dropna(inplace=True)
        self.s.reset_index(drop=True, inplace=True)
        self.s = self.s.astype(str)

    @cached_property
    def transactions(self) -> pd.DataFrame:
        transactions = []
        for t in self._raw_transactions:
            transactions.append(self._process_transaction(t))

        df = pd.DataFrame(transactions)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['Debit'] = pd.to_numeric(df['Debit'])
        df['Credit'] = pd.to_numeric(df['Credit'])

        return df

    def to_excel(self, filename=None, add_totals=True):
        if not filename:
            date = self.transactions['Date'].min()
            filename = f"{date:%m.%y}"

        table = self.transactions
        if add_totals:
            table = table.append(self.totals, ignore_index=True)
        
        writer = pd.ExcelWriter(f'Export/{filename}.xlsx', datetime_format='DD.MM.YY')

        with writer as w:
            table.to_excel(w, index=False, freeze_panes=(1, 0))

            ws = w.sheets['Sheet1']
            ws.set_column('A:Z', 20)
            ws.set_column('H:H', 40)
            ws.autofilter(0, 0, 0, 9)

    @cached_property
    def accounts(self) -> pd.Series:
        filt = self.s.str.contains('Счет No')
        accounts = self.s[filt]
        accounts = accounts.apply(lambda x: x.removeprefix('Счет No '))
        return accounts

    @cached_property
    def dates(self) -> pd.Series:
        filt = self.s.str.contains('Дата')
        dates = self.s[filt].apply(lambda x: x.split(' ')[2])
        return dates

    @property
    def totals(self) -> dict:
        # It would be easier to just calculate the totals,
        # but let's also extract them from the statement
        
        filt1 = self.s.str.contains('ИТОГО')
        filt2 = self.s.str.contains('за день')
        totals = self.s[filt1 & ~filt2]
        totals = totals.apply(self._remove_thousands_separator)

        debit_series = totals.apply(lambda x: x.split(' ')[5]).astype(float)
        credit_series = totals.apply(lambda x: x.split(' ')[3]).astype(float)

        total_debit = debit_series.sum() / 2
        total_credit = credit_series.sum() / 2

        return {'Debit': total_debit, 'Credit': total_credit}

    @property
    def _raw_transactions(self) -> list[pd.Series]:
        filt = self.s.str.contains('Контрагент: ')
        transaction_row_indices = [
            [i - 2, i - 1, i, i + 1, i + 2] for i in self.s[filt].index
        ]
        raw_transactions = [self.s.iloc[i] for i in transaction_row_indices]
        return raw_transactions

    def _process_transaction(self, t: pd.Series) -> dict:
        date = self.dates[self.dates.index < t.index[0]].iloc[-1]
        client_account = self.accounts[self.accounts.index < t.index[0]].iloc[-1]

        first_row = t.iloc[0]
        first_row = self._remove_thousands_separator(first_row)
        first_row = first_row.split(' ')

        payment_order = first_row[1] + ' ' + first_row[3]
        contractor_account = first_row[5] + ' ' + first_row[7]
        debit = first_row[15]
        credit = first_row[11]

        contractor_bank = t.iloc[1].split(': ')[1]
        contractor = t.iloc[2].split(': ')[1]
        contractor_unp = t.iloc[3].split(': ')[1]

        # some payment purpuse strings contain ':' symbol
        payment_purpose = t.iloc[4].split(': ')[1:]
        payment_purpose = ': '.join(payment_purpose)

        transaction = {'Date': date, 'Contractor': contractor, 'ContractorUnp': contractor_unp,
                       'PaymentOrder': payment_order, 'ContractorAccount': contractor_account,
                       'ContractorBank': contractor_bank, 'PaymentPurpose': payment_purpose,
                       'ClientAccount': client_account, 'Debit': debit, 'Credit': credit}

        return transaction

    @staticmethod
    def _remove_thousands_separator(row: str) -> str:
        regex = "\d{1,2}\s\d{3}[.]\d{2}"  # noqa
        return re.sub(regex, lambda match: match.group().replace(' ', ''), row)


Now it's time to merge the processed statements into a single DataFrame

In [24]:
class MergedStatement(Statement):
    
    def __init__(self, directory: str):
        self.files = glob.glob(os.path.join(directory, "*"))
    
    @property
    def transactions(self) -> pd.DataFrame:
        transactions = []
        for file in self.files:
            transactions.append(Statement(file).transactions)
        df = pd.concat(transactions)
        df.sort_values(by='Date', inplace=True)
        return df

    def to_excel(self):
        super().to_excel('Merged_Test', add_totals=False)
